In [1]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time
import copy
from tqdm import tqdm
import csv
import os
from torch.utils.data import Dataset
from PIL import Image

In [2]:
# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet mean/std
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Load datasets
data_dir = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}",
                                          transform=data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=128,
                             shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Load pre-trained ResNet50 and modify the final layer
model_ft = models.resnet50(pretrained=True)

# Freeze all layers
for param in model_ft.parameters():
    param.requires_grad = False

# Unfreeze the final block (optional fine-tuning)
for param in model_ft.layer4.parameters():
    param.requires_grad = True

# Replace the fully connected layer
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gpl/anaconda3/envs/VisualRecog/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Load the Model
model_path = 'resnet50_finetuned100.pth'
model_ft.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [7]:
# Total number of parameters
total_params = sum(p.numel() for p in model_ft.parameters())

# Number of trainable parameters
trainable_params = sum(p.numel() for p in model_ft.parameters()
                       if p.requires_grad)

print(f'Total parameters: {total_params}')
print(f'Trainable parameters: {trainable_params}')

Total parameters: 23712932
Trainable parameters: 15169636


In [5]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Only parameters that require gradients are optimized
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad,
                                       model_ft.parameters()), lr=1e-4)

# LR scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
# Train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        print('-' * 20)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Wrap dataloader with tqdm
            progress_bar = tqdm(dataloaders[phase],
                                desc=f'{phase.capitalize()} Phase',
                                leave=False)

            for inputs, labels in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize (training phase)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Stats
                batch_size = inputs.size(0)
                running_loss += loss.item() * batch_size
                running_corrects += torch.sum(preds == labels.data)

                # Update tqdm bar with loss and accuracy for the batch
                current_loss = running_loss / (running_corrects + 1e-8)
                current_acc = running_corrects.double() /\
                    (dataset_sizes[phase])
                progress_bar.set_postfix({'Loss': f'{current_loss:.4f}',
                                          'Acc': f'{current_acc:.4f}'})

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f}\
                  Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m\
          {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
# Training
model_ft = train_model(model_ft, criterion, optimizer_ft,
                       exp_lr_scheduler, num_epochs=100)


Epoch 1/100
--------------------


Train Loss: 2.2423 Acc: 0.5168


Val Loss: 1.7630 Acc: 0.5600

Epoch 2/100
--------------------


Train Loss: 1.0351 Acc: 0.7487


Val Loss: 1.1735 Acc: 0.6667

Epoch 3/100
--------------------


...

Val Loss: 0.5974 Acc: 0.8533

Epoch 99/100
--------------------


Train Loss: 0.2844 Acc: 0.9254


Val Loss: 0.5984 Acc: 0.8500

Epoch 100/100
--------------------


Train Loss: 0.2763 Acc: 0.9271


Val Loss: 0.6035 Acc: 0.8433

Training complete in 170m 15s
Best val Acc: 0.8567


In [8]:
# Save the trained model
torch.save(model_ft.state_dict(), 'resnet50_finetuned100.pth')

In [6]:
# Load the trained model
model = model_ft
model.load_state_dict(torch.load('resnet50_finetuned100.pth'))
model = model.to(device)
model.eval()

# Define the transform (same as training/validation)
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

val_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/val/'
val_dataset = datasets.ImageFolder(val_folder, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,
                                         shuffle=False)

correct = 0
total = 0
all_predictions = []

with torch.no_grad():
    for i, (inputs, labels) in enumerate(tqdm(val_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        true_label = labels.item()
        predicted_label = preds.item()

        correct += (predicted_label == true_label)
        total += 1

        # Get the file path of the current sample
        img_path, _ = val_dataset.samples[i]
        all_predictions.append((img_path.split('.')[0], predicted_label))

# Save predictions to a CSV
with open('val_predictions.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(all_predictions)


accuracy = correct / total * 100
print(f'Validation Accuracy: {accuracy:.2f}%')

100%|██████████| 300/300 [00:04<00:00, 67.47it/s]

Validation Accuracy: 85.67%


In [7]:
# Load the Model
model_path = 'resnet50_finetuned100.pth'
model_ft.load_state_dict(torch.load(model_path))
model = model_ft.to(device)
model.eval()

# Transforms (same as val transforms)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and Dataloader for test set
test_folder = '/home/gpl/文件/Kezia/Visual Recognition/HW1/hw1-data/data/test'

class TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path)
                            if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

# Create the dataset and loader
test_dataset = TestDataset(test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          shuffle=False)

# Run Inference on the Test Set
predictions = []

with torch.no_grad():
    for inputs, img_names in tqdm(test_loader, desc="Testing"):
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        predicted_label = preds.item()
        predicted_class = class_names[predicted_label]

        # Save predictions
        predictions.append((os.path.basename(img_names[0]).split('.')[0],
                            predicted_class))

predictions.sort(key=lambda x: x[0])

# Save predictions to a CSV
with open('prediction.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'pred_label'])  # header
    writer.writerows(predictions)

print("Predictions for test set saved to prediction.csv")

Testing: 100%|██████████| 2344/2344 [00:33<00:00, 70.37it/s]

Predictions for test set saved to prediction.csv
